In [ ]:
!pip install transformers
!pip install streamlit
!pip install pyngrok

!ngrok authtoken 1nwxinyHstfnSOTegWrAalPPy0j_7rPiqivYZwFJ45oHQQQwx

In [ ]:
!nvidia-smi

Wed Feb  3 20:55:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%writefile app.py
import streamlit as st
import os
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, pipeline
import tensorflow as tf
import time as t


tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = """The Campos Basin is one of 12 coastal sedimentary basins of Brazil. It spans both onshore and offshore parts of the South Atlantic with the onshore part located near Rio de Janeiro. The basin originated in Neocomian stage of the Cretaceous period 145–130 million years ago during the breakup of Gondwana. It has a total area of about 115,000 square kilometres (44,000 sq mi), with the onshore portion small at only 500 square kilometres (190 sq mi).
The basin is named after the Campos dos Goytacazes city.
The Campos Basin is bound on the south by the Cabo Frio High, separating the basin from the Santos Basin and on the north by the Vitória High, forming the boundary with the Espírito Santo Basin. Campos Basin contains the Paraiba do Sul River delta.
The South Atlantic margin developed on Archean stable cratons consisting of hard and resistant rocks and partly on the Neoproterozoic mobile belts composed of less resistant metamorphic rocks. The Precambrian basement of the Santos Basin is exposed as the Araçuaí Belt along the Brazilian coast, most notably in the inselbergs of Rio de Janeiro, of which Sugarloaf Mountain is the most iconic. The ancient rocks consist of a Neoproterozoic to Cambrian high-grade metamorphic core of granites and gneisses, formed during the collision of Gondwana in the Pan-African-Brasiliano orogeny.[5] Basalts similar to the Paraná and Etendeka traps, exposed to the west in the Paraná Basin, have been found underlying the Santos Basin. The Tristan da Cunha hotspot, known as the Tristan hotspot, is considered the driver behind the formation of these flood basalts.
"""

def bot_answer(question, text):
    '''Extractive QA.'''
    
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    outputs = model(inputs)

    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]  # Get the most likely beginning of answer with the argmax of the score
    answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]  # Get the most likely end of answer with the argmax of the score
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
    return answer

def simpler_bot_answer(question, text):
    '''Extractive QA using HF pipelines.'''

    tic = t.time()

    nlp = pipeline('question-answering')
    answer = nlp(question=question, context=text)

    tac = t.time()
    duration = round(tac - tic, 2)

    return answer, duration


# Streamlit
st.title('Blue Amazon Bot')
st.subheader('Ask anything about the Blue Amazon region')

qa_algorithm = st.selectbox('Choose the algorithm to be used for question answering:', ('BERT', 'T5', 'RAG'))
question = st.text_input('What do you want to know about the Blue Amazon region?')

# st.write('BOT 1) The best answer is:', bot_answer(question, text))
if question:
    answer, duration = simpler_bot_answer(question, text)
    st.write('The best answer is: {} (score: {} | {}s)'.format(answer['answer'], round(answer['score'], 2), duration))

# st.write('Libs instaladas', os.popen('pip freeze').read())
st.text('')
st.write('Is GPU available?  \n  \n', os.popen('nvidia-smi').read())


Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(addr='8501')
print(public_url)

NgrokTunnel: "http://e1f0370bf37a.ngrok.io" -> "http://localhost:8501"


In [ ]:
ngrok.kill()